In [ ]:
#  import os
#  import sys

#  current_dir = os.getcwd()
#  project_root = os.path.abspath(os.path.join(current_dir, "..","..","..", "src"))
#  sys.path.append(project_root)

In [4]:
print(sys.path)

['C:\\Users\\icero\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\icero\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\icero\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\icero\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\icero\\Downloads\\CICD Db DAB\\dab_project\\.venv_dbc', '', 'c:\\Users\\icero\\Downloads\\CICD Db DAB\\dab_project\\.venv_dbc\\Lib\\site-packages', 'c:\\Users\\icero\\Downloads\\CICD Db DAB\\dab_project\\.venv_dbc\\Lib\\site-packages\\setuptools\\_vendor', 'c:\\Users\\icero\\Downloads\\CICD Db DAB\\dab_project', 'c:\\Users\\icero\\Downloads\\CICD Db DAB\\dab_project\\src']


In [ ]:
# #os.getcwd()
# project_root = os.path.abspath(os.path.join(current_dir, "..","..",".."))
# print(project_root)

c:\Users\icero\Downloads\CICD Db DAB\dab_project


In [5]:
from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [ ]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [ ]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [ ]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [ ]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [ ]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")